<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Trade_AI/blob/main/exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729164 entries, 0 to 729163
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   origin       729164 non-null  string        
 1   destination  729164 non-null  string        
 2   hs6          729164 non-null  string        
 3   hs4          729164 non-null  string        
 4   trade_flow   729164 non-null  string        
 5   month        729164 non-null  datetime64[ns]
 6   value        729164 non-null  int64         
dtypes: datetime64[ns](1), int64(1), string(5)
memory usage: 38.9 MB


,origin,destination,hs6,hs4,trade_flow,month,value
0,USA,ABW,020130,0201,Exports,2025-01-01,92691
1,USA,ABW,020230,0202,Exports,2025-01-01,72148
2,USA,ABW,020319,0203,Exports,2025-01-01,18846
3,USA,ABW,020329,0203,Exports,2025-01-01,12639
4,USA,ABW,020442,0204,Exports,2025-01-01,3426


In [3]:
import pandas as pd
path = "/content/drive/MyDrive/ai4trade/USA_2025.parquet"  # <- your file

df = pd.read_parquet(path)

# Keep/standardize expected columns & dtypes
expected = ['origin','destination','hs6','hs4','trade_flow','month','value']
df = df[[c for c in expected if c in df.columns]].copy()

df['origin'] = df['origin'].astype('string')
df['destination'] = df['destination'].astype('string')
df['hs6'] = df['hs6'].astype('string')
df['hs4'] = df['hs4'].astype('string')
df['trade_flow'] = df['trade_flow'].astype('string')
df['value'] = pd.to_numeric(df['value'], errors='coerce').fillna(0).astype('int64')
df['month'] = pd.to_datetime(df['month'], errors='coerce')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729164 entries, 0 to 729163
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   origin       729164 non-null  string        
 1   destination  729164 non-null  string        
 2   hs6          729164 non-null  string        
 3   hs4          729164 non-null  string        
 4   trade_flow   729164 non-null  string        
 5   month        729164 non-null  datetime64[ns]
 6   value        729164 non-null  int64         
dtypes: datetime64[ns](1), int64(1), string(5)
memory usage: 38.9 MB


,origin,destination,hs6,hs4,trade_flow,month,value
0,USA,ABW,020130,0201,Exports,2025-01-01,92691
1,USA,ABW,020230,0202,Exports,2025-01-01,72148
2,USA,ABW,020319,0203,Exports,2025-01-01,18846
3,USA,ABW,020329,0203,Exports,2025-01-01,12639
4,USA,ABW,020442,0204,Exports,2025-01-01,3426


In [4]:
print("Rows:", len(df))
print("Date range:", df['month'].min(), "→", df['month'].max())
print("Flows:", df['trade_flow'].unique().tolist())
print("Unique destinations:", df['destination'].nunique())
print("Negative values:", (df['value'] < 0).sum())

# Duplicates at the intended grain (month, destination, hs6, flow)
dupes = df.duplicated(subset=['month','destination','hs6','trade_flow']).sum()
print("Potential duplicates @ (month,destination,hs6,trade_flow):", dupes)

# If needed, ensure uniqueness by summing:
# df = df.groupby(['month','destination','hs6','hs4','trade_flow'], as_index=False)['value'].sum()


Rows: 729164
Date range: 2025-01-01 00:00:00 → 2025-02-01 00:00:00
Flows: ['Exports', 'Imports']
Unique destinations: 228
Negative values: 0
Potential duplicates @ (month,destination,hs6,trade_flow): 270641


In [5]:
# Overall monthly totals
m_total = (df.groupby('month', as_index=False)['value']
             .sum()
             .rename(columns={'value':'total_value'}))

# By flow (Export/Import)
m_flow = df.groupby(['month','trade_flow'], as_index=False)['value'].sum()
m_flow_piv = m_flow.pivot(index='month', columns='trade_flow', values='value').sort_index()

m_total.tail(), m_flow_piv.tail()


(       month   total_value
 0 2025-01-01  481331908570
 1 2025-02-01  455795952915,
 trade_flow       Exports       Imports
 month                                 
 2025-01-01  164025679587  317306228983
 2025-02-01  167608819040  288187133875)

In [7]:
# Top 10 export destinations
top10_exp_dest = (df[df['trade_flow'] == 'Exports']
                  .groupby('destination', as_index=False)['value']
                  .sum()
                  .nlargest(10, 'value'))

# Top 10 import destinations
top10_imp_dest = (df[df['trade_flow'] == 'Imports']
                  .groupby('destination', as_index=False)['value']
                  .sum()
                  .nlargest(10, 'value'))

print("Top 10 Export Destinations:")
display(top10_exp_dest)

print("\nTop 10 Import Destinations:")
display(top10_imp_dest)

Top 10 Export Destinations:


,destination,value
33,CAN,54774972028
128,MEX,54683096785
37,CHN,20362940740
70,GBR,14960717656
150,NLD,14030540634
101,JPN,12415155223
52,DEU,12061242523
108,KOR,10608349665
66,FRA,7958227570
27,BRA,7928810824



Top 10 Import Destinations:


,destination,value
129,MEX,83317330939
37,CHN,73274542996
33,CAN,73263168668
35,CHE,43759309120
94,IRL,28910815048
52,DEU,26338644896
220,VNM,24906421597
102,JPN,23942171046
210,TWN,21657972614
109,KOR,19686069338


In [8]:
# Top 10 exported items (using hs6)
top10_hs6_exp = (df[df['trade_flow'] == 'Exports']
                 .groupby('hs6', as_index=False)['value']
                 .sum()
                 .nlargest(10, 'value'))

# Top 10 imported items (using hs6)
top10_hs6_imp = (df[df['trade_flow'] == 'Imports']
                 .groupby('hs6', as_index=False)['value']
                 .sum()
                 .nlargest(10, 'value'))

print("Top 10 Exported Items (HS6):")
display(top10_hs6_exp)

print("\nTop 10 Imported Items (HS6):")
display(top10_hs6_imp)

Top 10 Exported Items (HS6):


,hs6,value
4903,880330,21791592313
993,270900,18413710892
5329,999999,10607479312
995,271019,9593896372
994,271012,7456500469
999,271111,7313423331
4741,854231,6040493644
3360,710812,5976072202
1562,300490,4381128730
1000,271112,4234469288



Top 10 Imported Items (HS6):


,hs6,value
3394,711590,55164264070
997,270900,25230960555
5328,999999,21526735470
1494,293719,16900631047
1531,300215,14291564393
1553,300490,14221965060
4396,847150,12605661140
4821,870323,11910655926
4406,847330,11638558361
4603,851713,10328961844


In [10]:
# Build a tidy series at the (flow,destination,hs6,month) grain
series = (df.groupby(['trade_flow','destination','hs6','month'], as_index=False)['value']
            .sum().sort_values('month'))

# Pick the top Export series by total value
anchor = (series[series['trade_flow']=='Exports'] # Changed 'Export' to 'Exports'
          .groupby(['destination','hs6'], as_index=False)['value'].sum()
          .sort_values('value', ascending=False).head(1)[['destination','hs6']].iloc[0])

dst, h6 = anchor['destination'], anchor['hs6']
one = series[(series['trade_flow']=='Exports') & (series['destination']==dst) & (series['hs6']==h6)].copy() # Changed 'Export' to 'Exports'
one = one.sort_values('month')

for L in [1,2,3,6,12]:
    one[f'lag_{L}'] = one['value'].shift(L)
one['ma_3'] = one['value'].rolling(3).mean()
one['ma_6'] = one['value'].rolling(6).mean()

one.tail(12)

,trade_flow,destination,hs6,month,value,lag_1,lag_2,lag_3,lag_6,lag_12,ma_3,ma_6
109794,Exports,GBR,710812,2025-01-01,436202447,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109795,Exports,GBR,710812,2025-02-01,3494262095,436202447.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.head()


,origin,destination,hs6,hs4,trade_flow,month,value
0,USA,ABW,020130,0201,Exports,2025-01-01,92691
1,USA,ABW,020230,0202,Exports,2025-01-01,72148
2,USA,ABW,020319,0203,Exports,2025-01-01,18846
3,USA,ABW,020329,0203,Exports,2025-01-01,12639
4,USA,ABW,020442,0204,Exports,2025-01-01,3426
